In [ ]:
import ast
import json
import copy
import pandas as pd
import numpy as np
import random
import os

# Dataset Generation

In [ ]:
def generate_dataset_webnlg(pair_src, pair_tgt, graph, text):
    wf_src = open(pair_src, 'a')
    wf_tgt = open(pair_tgt, 'a')
    
    # y = 'Correct'
    # wf_src.write(text + ' <S> ' + json.dumps(graph, ensure_ascii=False) + '\n')
    # wf_tgt.write(y + '\n')
    
    for i in range(len(graph)):
        x = copy.deepcopy(graph)

        if len(graph) != 1:
            y = x[i]
            x.pop(i)
            # x[i][0], x[i][2] = x[i][2], x[i][0]
            wf_src.write(text + ' <S> ' + json.dumps(x, ensure_ascii=False) + '\n')
            wf_tgt.write(json.dumps(y, ensure_ascii=False) + '\n')

    wf_tgt.close()
    wf_src.close()

In [ ]:
def generate_dataset_kelm(pair_src, pair_tgt, graph, text):
    wf_src = open(pair_src, 'a')
    wf_tgt = open(pair_tgt, 'a')
    
    y = 'Correct'
    wf_src.write(text + ' <S> ' + json.dumps(graph, ensure_ascii=False) + '\n')
    wf_tgt.write(y + '\n')
    
    x = copy.deepcopy(graph)
    if len(graph) != 1:
        y = random.choice(x)
        x.pop(x.index(y))
        wf_src.write(text + ' <S> ' + json.dumps(x, ensure_ascii=False) + '\n')
        wf_tgt.write(json.dumps(y, ensure_ascii=False) + '\n')

    wf_tgt.close()
    wf_src.close()

In [ ]:
def generate_dataset_unseen(pair_src, pair_tgt, graph, text):
    wf_src = open(pair_src, 'a')
    wf_tgt = open(pair_tgt, 'a')
    
    # y = 'Correct'
    # wf_src.write(text + ' <S> ' + json.dumps(graph, ensure_ascii=False) + '\n')
    # wf_tgt.write(y + '\n')
    
    for i in range(len(graph)):
        x = copy.deepcopy(graph)
        
        if len(graph) != 1:
            if x[i][1][2:].strip().lower() not in text.lower() and x[i][2][2:].strip().lower() not in text.lower():
                y = x[i][:-1]
                x.pop(i)
                wf_src.write(text + ' <S> ' + json.dumps(x, ensure_ascii=False) + '\n')
                wf_tgt.write(json.dumps(y, ensure_ascii=False) + '\n')

    wf_tgt.close()
    wf_src.close()

In [ ]:
def generate_dataset_correct(pair_src, pair_tgt, graph, text):
    wf_src = open(pair_src, 'a')
    wf_tgt = open(pair_tgt, 'a')
    
    y = 'Correct'
    wf_src.write(text + ' <S> ' + json.dumps(graph, ensure_ascii=False) + '\n')
    wf_tgt.write(y + '\n')

    wf_tgt.close()
    wf_src.close()

In [ ]:
pair_train_src = "data/only_one_error_webnlg/train.source"
pair_train_tgt = "data/only_one_error_webnlg/train.target"

with open("webnlg20/train.source", 'r') as fa, open('webnlg20/train.target', 'r') as fb:
    a = fa.readlines()
    b = fb.readlines()
    existing_graphs = []
    for i in range(len(a)):
        # change string to list
        graph = ast.literal_eval(a[i].strip())
        text = b[i].strip()
        generate_dataset_correct(pair_train_src, pair_train_tgt, graph, text)
        if graph not in existing_graphs:
            generate_dataset_webnlg(pair_train_src, pair_train_tgt, graph, text)
            # generate_dataset_unseen(pair_train_src, pair_train_tgt, graph, text)
            existing_graphs.append(graph)

In [ ]:
pair_train_src = "data/only_one_error_kelm/train.source"
pair_train_tgt = "data/only_one_error_kelm/train.target"

with open("data/kelm_subset/train.source", 'r') as fa, open('data/kelm_subset/train.target', 'r') as fb:
    a = fa.readlines()
    b = fb.readlines()
    for i in range(len(a)):
        # change string to list
        graph = ast.literal_eval(a[i].strip())
        text = b[i].strip()
        generate_dataset_kelm(pair_train_src, pair_train_tgt, graph, text) 

# Post-Processing

In [ ]:
# Create unified verifier input
all_data = []
with open("GPT3.5_result_KELM_6_shots/Iteration1/test_generated_graphs.txt", 'r') as fa, open('GPT3.5_result_KELM_1_shots/test.target', 'r') as fb:
    a = fa.readlines()
    b = fb.readlines()
    for i in range(len(a)):
        data_dict = {}
        graph = a[i].strip()
        text = b[i].strip()
        data_dict["instruction"] = "Predict the missing triple given the text and graph for KELM dataset."
        data_dict["input"] = text.replace('"',"'") + ' <S> ' + str(graph).replace('"',"'") 
        all_data.append(data_dict)
with open('GPT3.5_result_KELM_6_shots/Iteration3/verifier_input.json','w',encoding='utf-8') as file:
    file.write(json.dumps(all_data, indent=4, ensure_ascii=False))

In [ ]:
# Create single verifier input
test_generated_graphs = []
test_texts = []
with open("GPT3.5_result_GenWiki/Iteration1/test_generated_graphs.txt", 'r') as f:
    for line in f.readlines():
        test_generated_graphs.append(line.strip())

with open("GPT3.5_result_GenWiki/test.target", 'r') as f:
    for line in f.readlines():
        test_texts.append(line.strip())

with open("GPT3.5_result_GenWiki/Iteration1/test.source", 'w') as f:
    for i in range(len(test_generated_graphs)):
        f.write(test_texts[i] + ' <S> ' + test_generated_graphs[i] + '\n')

In [ ]:
# Iterative Ofﬂine Correction
full_test_generated_graphs = []
full_verifier_texts = []
with open("GPT3.5_result_KELM_6_shots/Iteration3/test_generated_graphs.txt", 'r') as f:
    for line in f.readlines():
        full_test_generated_graphs.append(line.strip())

with open("GPT3.5_result_KELM_6_shots/Iteration3/verifier_result.txt", 'r') as f:
    for line in f.readlines():
        full_verifier_texts.append(line.strip())

with open("GPT3.5_result_KELM_6_shots/Iteration4/test_generated_graphs.txt", 'w') as f:
    for i in range(len(full_test_generated_graphs)):
        if full_verifier_texts[i] != 'Correct':
            f.write(full_test_generated_graphs[i][:-1] + ', ' + full_verifier_texts[i] + ']' + '\n')
        else:
            f.write(full_test_generated_graphs[i] + '\n')

# Calculate Accuracy

In [ ]:
def cal_accuracy(verification_results):
    correct_num = 0

    for i in range(len(verification_results)):
        if verification_results[i] == 'Correct':
            correct_num += 1

    print('Accuracy: ', correct_num/len(verification_results))    
    return correct_num/len(verification_results)